### TEST for 2 Atoms -> influence of angle to V and $\Gamma$

In [20]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
from matplotlib.animation import FuncAnimation
import os

In [ ]:
save_path = r'C:\Users\leopo\OneDrive - UT Cloud\Uni\Semester_8\BA_mit_Git\BA_Plots\Qutip'
os.makedirs(os.path.dirname(save_path), exist_ok=True)

In [22]:
########################################                 Define constants                   #############################################
N = 2
fixed_lam   = 1
k_a = 2 * np.pi / fixed_lam
fixed_gamma = 1

fixed_dist  = 0.235 * fixed_lam
fixed_phi   = np.pi/2

x_0      = 0.02
x_max    = 1.4
x_size = 200

phi_0    = 0
phi_max  = np.pi / 2
phi_size = 4

t_max    = 40
t_size   = 100
########################################################################################################################    
times  = np.linspace(0, t_max, t_size) * fixed_gamma
dists  = np.linspace(x_0, x_max, x_size) * fixed_lam
phis   = np.linspace(phi_0, phi_max, phi_size)

In [23]:
########################################               define the geometry                 #############################################
def dipole_vector(phi = fixed_phi):
    """Returns the dipole vector given its orientation by angle phi."""
    dipole = np.array([np.cos(phi), np.sin(phi), 0])
    return dipole

def chain_positions(distance = fixed_dist, N = N):
    Pos = np.zeros((N, 3))
    for i in range(N):
        Pos[i,0] = i * distance
    return Pos

In [24]:
########################################             create the Hamiltonean                 #################################################
def Green_tensor(r_a, r_b, k_a):
    r_ab = r_b - r_a
    abs_r_ab = np.linalg.norm(r_ab)
    kappa = k_a * abs_r_ab
    return (np.exp(1j * kappa) / (4 * np.pi * kappa ** 2 * abs_r_ab)
                    * ((kappa ** 2 + 1j * kappa - 1) * np.eye(3)
                       + (- kappa ** 2 - 3 * 1j * kappa + 3)
                       * np.outer(r_ab, r_ab) / (abs_r_ab ** 2)))

def Gamma_matrix(distance, dipoles, lam, gamma):
    positions = chain_positions(distance)
    G_matrix = np.zeros((N, N), dtype=complex)
    for a in range(N):
        for b in range(N):
            G_matrix[a, b] = gamma
            r_a, r_b = positions[a], positions[b]
            if np.linalg.norm(r_b - r_a) > 1e-5:
                d_a, d_b = dipoles[a], dipoles[b]
                k_a = 2 * np.pi / lam
                result = (6 * np.pi * gamma / k_a * np.matmul(np.conj(d_a), np.matmul(Green_tensor(r_a, r_b, k_a), d_b.T)))
                G_matrix[a, b] = np.imag(result)
    return G_matrix

def V_matrix(distance, dipoles, lam, gamma):
    positions = chain_positions(distance, N)
    V_matrix = np.zeros((N, N), dtype=complex)
    for a in range(N):
        for b in range(N):
            r_a, r_b = positions[a], positions[b]
            V_matrix[a, b] = 0
            if np.linalg.norm(r_b - r_a) > 1e-5:
                d_a, d_b = dipoles[a], dipoles[b]
                k_a = 2 * np.pi / lam
                result = (3 * np.pi * gamma / k_a * np.matmul(np.conj(d_a), np.matmul(Green_tensor(r_a, r_b, k_a), d_b.T)))
                V_matrix[a, b] = np.real(result)
    return V_matrix
    
def H_eff(distance = fixed_dist, dipoles = [dipole_vector() for _ in range(N)], lam=fixed_lam, gamma=fixed_gamma):
    G = Gamma_matrix(distance, dipoles, lam, gamma)
    V = V_matrix(distance, dipoles, lam, gamma)
    return Qobj(V) - 1j / 2 * Qobj(G)

In [39]:
Gamma_vals = np.zeros((x_size, phi_size))
V_vals = np.zeros((x_size, phi_size))

for i, dist in enumerate(dists):
    for j, phi in enumerate(phis):
        dipoles = [dipole_vector(phi) for _ in range(N)]
        G = Gamma_matrix(dist, dipoles, fixed_lam, fixed_gamma)
        V = V_matrix(dist, dipoles, fixed_lam, fixed_gamma)
        Gamma_vals[i, j] = G[0, 1]
        V_vals[i, j] = V[0, 1]

In [41]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

for j in range(phi_size):
    ax1.plot(dists / fixed_lam, V_vals[:, j] / fixed_gamma, label=f'$\\phi = {round(phis[j], 2)}$')
    ax2.plot(dists / fixed_lam, Gamma_vals[:, j] / fixed_gamma, label=f'$\\phi = {round(phis[j], 2)}$')

ax1.set_xlabel('$a/\\lambda$')
ax1.set_ylabel('$V_{12}(\\phi, a) / \Gamma_0$')
ax1.set_ylim([-1, 1])  # Limit y-axis
ax1.legend()
ax1.grid(True)

ax2.set_xlabel('$a/\\lambda$')
ax2.set_ylabel('$\Gamma_{12}(\\phi, a) / \Gamma_0$')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.savefig(save_path + '2_atoms_Matrix_elems_same_dipoles(r).svg')
plt.show()

In [51]:
# Define fixed phi values for each plot
fixed_phis = [0, np.pi/6, np.pi/3, np.pi/2, 2*np.pi/3, 5*np.pi/6, np.pi]
titles = [
    'Fixed phi = 0', 'Fixed phi = π/6', 'Fixed phi = π/3', 
    'Fixed phi = π/2', 'Fixed phi = 2π/3', 'Fixed phi = 5π/6', 'Fixed phi = π'
]

# Create subplots
fig, axes = plt.subplots(len(fixed_phis), 1, figsize=(5, 24))

# Get default color cycle
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

for idx, fixed_phi in enumerate(fixed_phis):
    Gamma_vals = np.zeros((x_size, phi_size))
    V_vals = np.zeros((x_size, phi_size))

    # Iterate over distances and varying phi angles
    for i, dist in enumerate(dists):
        for j, phi in enumerate(phis):
            dipoles = [dipole_vector(fixed_phi)] + [dipole_vector(phi) for _ in range(1, N)]  # First dipole fixed, rest vary
            G = Gamma_matrix(dist, dipoles, fixed_lam, fixed_gamma)
            V = V_matrix(dist, dipoles, fixed_lam, fixed_gamma)
            Gamma_vals[i, j] = G[0, 1]  # Store interaction of first two atoms for simplicity
            V_vals[i, j] = V[0, 1]  # Store interaction of first two atoms for simplicity

    ax = axes[idx]
    for j in range(phi_size):
        # Use default colors cycling through them
        color = colors[j % len(colors)]
        ax.plot(dists / fixed_lam, V_vals[:, j] / fixed_gamma, color=color, label=f'$V: \\phi = {round(phis[j], 2)}$')
        ax.plot(dists / fixed_lam, Gamma_vals[:, j] / fixed_gamma, '--', color=color, label=f'$\\Gamma: \\phi = {round(phis[j], 2)}$')

    ax.set_xlabel('$a/\\lambda$')
    ax.set_ylabel('$V_{12}, \\Gamma_{12} / \Gamma_0$')
    ax.set_ylim([-1, 1])  # Limit y-axis
    ax.set_title(titles[idx])
    ax.grid(True)

# Add legends to the first subplot
handles, labels = axes[0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
axes[0].legend(by_label.values(), by_label.keys(), loc='best')

plt.tight_layout()
plt.savefig(save_path + '2_atoms_Matrix_elems_1_dipole_fix(r).svg')
plt.show()

In [66]:
fixed_phis = [0, np.pi/6, np.pi/3, np.pi/2, 2*np.pi/3, 5*np.pi/6]
titles = [
    'Fixed phi = 0', 'Fixed phi = π/6', 'Fixed phi = π/3', 
    'Fixed phi = π/2', 'Fixed phi = 2π/3', 'Fixed phi = 5π/6']

# Create subplots in a 1x2 grid layout
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Get default color cycle
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

# Plot for V values
ax_v = axes[0]
for idx, fixed_phi in enumerate(fixed_phis):
    V_vals = np.zeros((x_size, phi_size))
    for i, dist in enumerate(dists):
        for j, phi in enumerate(phis):
            dipoles = [dipole_vector(fixed_phi)] + [dipole_vector(phi) for _ in range(1, N)]  # First dipole fixed, rest vary
            V = V_matrix(dist, dipoles, fixed_lam, fixed_gamma)
            V_vals[i, j] = V[0, 1]  # Store interaction of first two atoms for simplicity

    for j in range(phi_size):
        color = colors[j % len(colors)]
        ax_v.plot(dists / fixed_lam, V_vals[:, j] / fixed_gamma, color=color, alpha=0.1 + 0.9*(1-(idx/len(fixed_phis))), label=f'$V: \\phi = {round(phis[j], 2)}$' if idx == 0 else "")

ax_v.set_xlabel('$a/\\lambda$')
ax_v.set_ylabel('$V_{12} / \Gamma_0$')
ax_v.set_ylim([-1, 1])
ax_v.set_title('V values')
ax_v.grid(True)

# Plot for Gamma values
ax_gamma = axes[1]
for idx, fixed_phi in enumerate(fixed_phis):
    Gamma_vals = np.zeros((x_size, phi_size))
    for i, dist in enumerate(dists):
        for j, phi in enumerate(phis):
            dipoles = [dipole_vector(fixed_phi)] + [dipole_vector(phi) for _ in range(1, N)]  # First dipole fixed, rest vary
            G = Gamma_matrix(dist, dipoles, fixed_lam, fixed_gamma)
            Gamma_vals[i, j] = G[0, 1]  # Store interaction of first two atoms for simplicity

    for j in range(phi_size):
        color = colors[j % len(colors)]
        ax_gamma.plot(dists / fixed_lam, Gamma_vals[:, j] / fixed_gamma, '--', color=color, alpha=0.1 + 0.9*(1-(idx/len(fixed_phis))), label=f'$\\Gamma: \\phi = {round(phis[j], 2)}$' if idx == 0 else "")

ax_gamma.set_xlabel('$a/\\lambda$')
ax_gamma.set_ylabel('$\\Gamma_{12} / \Gamma_0$')
ax_gamma.set_ylim([-1, 1])
ax_gamma.set_title('Gamma values')
ax_gamma.grid(True)

# Add a combined horizontal legend to the first subplot
handles, labels = ax_v.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), ncol=2, bbox_to_anchor=(0.5, -0.1))

plt.tight_layout()
#plt.savefig('2_atoms_Matrix_elems_1_dipole_fix(r).svg')
plt.show()

In [18]:
# Define fixed phi values for each plot
fixed_phis = [0, np.pi/6, np.pi/3, np.pi/2, 2*np.pi/3, 5*np.pi/6, np.pi]
titles = [
    'Fixed phi = 0', 'Fixed phi = π/6', 'Fixed phi = π/3', 
    'Fixed phi = π/2', 'Fixed phi = 2π/3', 'Fixed phi = 5π/6', 'Fixed phi = π'
]

# Create subplots in a 2x4 grid layout
fig, axes = plt.subplots(2, 4, figsize=(20, 10))

# Flatten the axes array for easy iteration
axes = axes.flatten()

for idx, fixed_phi in enumerate(fixed_phis):
    angles = np.zeros((x_size, phi_size))

    # Iterate over distances and varying phi angles
    for i, dist in enumerate(dists):
        for j, phi in enumerate(phis):
            dipoles = [dipole_vector(fixed_phi)] + [dipole_vector(phi) for _ in range(1, N)]  # First dipole fixed, rest vary
            G = Gamma_matrix(dist, dipoles, fixed_lam, fixed_gamma)
            V = V_matrix(dist, dipoles, fixed_lam, fixed_gamma)
            complex_value = V[0, 1] + 1j * G[0, 1] / 2
            angles[i, j] = np.angle(complex_value)  # Calculate the angle of the complex number

    ax = axes[idx]
    for j in range(phi_size):
        ax.plot(dists / fixed_lam, angles[:, j], label=f'$\\phi = {round(phis[j], 2)}$')

    ax.set_xlabel('$a/\\lambda$')
    ax.set_ylabel('Angle (radians)')
    ax.set_ylim([-np.pi, np.pi])  # Limit y-axis to -π to π
    ax.set_title(titles[idx])
    ax.grid(True)

# Add legends to the first subplot
handles, labels = axes[0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
axes[0].legend(by_label.values(), by_label.keys(), loc='best')

# Hide any unused subplots
for ax in axes[len(fixed_phis):]:
    fig.delaxes(ax)

plt.tight_layout()
plt.show()

In [ ]:
    def survival_probabilities(dipoles, times, distance=fixed_dist, gamma=fixed_gamma, Psi_0=basis(N, 0)):
mods = np.zeros((len(times), N))
    P_surs = np.zeros((len(times)))
    for t_idx, t in enumerate(times):
        U = (-1j * H * t).expm()
        Psi_t = (U * Psi_0).full().flatten()
        Probs = np.abs(Psi_t)**2

        coeffs_sq_mods[t_idx, :] = Probs
        P_surs[t_idx] = Probs.sum()

    return coeffs_sq_mods, P_surs

# Define the dipoles for both atoms having phi = pi/2
dipoles = [dipole_vector(np.pi / 2) for _ in range(N)]

# Calculate survival probabilities
psi0 = (basis(N, 0) + basis(N,1)).unit()
distance = 0.15 * fixed_lam
y1, y2 = survival_probabilities(dipoles, times, distance=distance, Psi_0=psi0)
positions = chain_positions(fixed_dist, N)

#
# TODO implement this everywhere
#
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))
# Plot the evolution of atom state probabilities
im = ax1.imshow(y1.T, aspect='auto', origin='lower', cmap='viridis', extent=[0, t_max, 0, N-1])
ax1.set_xlabel('Time')
ax1.set_ylabel('Atom index')
ax1.set_title('Time evolution of atom state probabilities')
fig.colorbar(im, ax=ax1, label='Probability')

# Plot atoms with their dipole moments
ax2.scatter(positions[:, 0], positions[:, 1], color='blue', s=100, label='Atoms')
for i in range(N):
    ax2.arrow(positions[i, 0], positions[i, 1], dipoles[i][0] * 0.01, dipoles[i][1] * 0.01, head_width=0.5 * 0.01, head_length=0.5 * 0.01, fc='red', ec='red')
ax2.set_title('Atom Positions and Dipole Moments')
ax2.set_xlabel('X Position')
ax2.set_ylabel('Y Position')
ax2.grid(True)
ax2.legend()
ax2.axis('equal')

plt.tight_layout()
plt.show()

# Plot the norms against time
plt.figure(figsize=(12, 2))
plt.plot(times, y2, label='Survival Probability')
plt.plot(times, np.exp(-times), 'r--', linewidth=2.5, label=r'exp(-$\gamma t$)')
plt.xlabel('Time') 
plt.ylabel('Norm (Survival Probability)')
plt.title('Survival Probability vs. Time for Dipole Configuration with $\phi = \pi/2$')
plt.legend()
plt.grid(True)
plt.show()